In [1]:
import os

In [2]:
%pwd

'/mnt/d/DeepLearning/Text_Summarizer_end-to-end_nlp_project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/d/DeepLearning/Text_Summarizer_end-to-end_nlp_project'

Update config.yaml ✅  
Update params.yaml ✅   
Update entity

In [11]:
from dataclasses import dataclass
from pathlib import Path

In [12]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

Update configuration manager

In [13]:
from text_summarizer.constants import *
from text_summarizer.utils.common import create_directories, read_yaml

In [18]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )

        return model_evaluation_config

Update the componenet file

In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

In [20]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """
            Split the dataset into smaller batches that can be processed simultaneously.
            Yield successive batch-sized chunks from list of elements
        """

        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, column_text='article',
                                    device='cuda' if torch.cuda.is_available() else 'cpu',
                                    column_summary='highlights'):
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding='max_length', return_tensor='pt')
            
            summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                     attention_mask = inputs['attention_mask'].to(device),
                                     length_penalty=0.8,
                                     num_beams=8,
                                     max_length=128)
            ''' parameter for length_penality ensures that the model does not generate sequences that are too long.'''

            # Finally, we decode the generated texts by replacing the token, and add the decoded
            # texts with references to the metric.

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)            
                                 for s in summaries]
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predcitions=decoded_summaries, references=target_batch)

        # Finally compute and return the ROUGE score
        score = metric.compute()

        return score
    

    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenzier = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # Loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
        rouge_metric = load_metric('rouge')
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][1:10], 
            rouge_metric, 
            model_pegasus, 
            tokenizer=tokenzier, 
            batch_size=2, 
            column_text='dialogue', 
            column_summary='summary'
        ) 

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)
    

Update pipeline

In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[ 2024-01-01 17:26:55,416] 36 root - INFO - common - Read yaml file: config/config.yaml
[ 2024-01-01 17:26:55,422] 36 root - INFO - common - Read yaml file: params.yaml
[ 2024-01-01 17:26:55,426] 59 root - INFO - common - Created directory at: artifacts


: 